In [ ]:
%%writefile install_mfa.sh
!/bin/bash
## a script to install Montreal Forced Aligner (MFA)
root_dir=${1:-/tmp/mfa}
mkdir -p $root_dir
cd $root_dir

# download miniconda3
wget -q --show-progress https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh -b -p $root_dir/miniconda3 -f

# download mfa aligner
$root_dir/miniconda3/bin/conda create -n aligner -c conda-forge montreal-forced-aligner -y
# source $root_dir/miniconda3/bin/conda activate aligner

echo -e "\n======== DONE =========="


In [ ]:
# download and install mfa
INSTALL_DIR="/tmp/mfa" # path to install directory
!bash ./install_mfa.sh {INSTALL_DIR}

In [ ]:
!echo "normalize audio clips to sample rate of 16k"
!find /kaggle/input/librispeech/Librispeech/19 -name "*.wav" -type f -execdir sox --norm=-3 {} -r 16k -c 1 `pwd`/wav/{} \;
!echo "Number of clips" $(ls ./wav/ | wc -l)

In [ ]:
# download a pretrained english acoustic model, and english lexicon
!wget -q --show-progress https://github.com/MontrealCorpusTools/mfa-models/raw/main/acoustic/english.zip
!wget -q --show-progress http://www.openslr.org/resources/11/librispeech-lexicon.txt

In [ ]:
# see: https://github.com/MontrealCorpusTools/Montreal-Forced-Aligner/pull/480
import re
lexicon = open("librispeech-lexicon.txt").readlines()
sp = re.compile("\s+")
with open("modified_librispeech-lexicon.txt", "w") as f:
    print(lexicon)
    for line in lexicon:
        word, *phonemes = sp.split(line.strip())
        phonemes = " ".join(phonemes)
        f.write(f"{word}\t{phonemes}\n")

In [ ]:
# FINALLY, align phonemes and speech
!source {INSTALL_DIR}/miniconda3/bin/activate aligner; \
mfa align -t ./temp -j 4 ./wav modified_librispeech-lexicon.txt ./english.zip ./ljs_aligned
# output files are at ./ljs_aligned
!echo "See output files at ./ljs_aligned"